In [214]:
import pandas as pd
import mysql.connector

In [215]:
df_at = pd.read_csv('datos/Limpieza-4.csv', index_col=0)
df_at.head()

,year,type,country,activity,age,species,month,fatal,sex
0,2018,Boating,usa,Paddling,57.0,White Shark,Jun,N,F
1,2018,Unprovoked,brazil,Swimming,18.0,Tiger Shark,Jun,Y,M
2,2018,Unprovoked,usa,Walking,15.0,Bull Shark,May,N,M
3,2018,Provoked,australia,Feeding sharks,32.0,Grey Shark,May,N,M
4,2018,Invalid,england,Fishing,21.0,Unspecified,May,N,M


In [216]:
list_country = ['usa', 'australia', 'south africa', 'new zealand', 'papua new guinea']

In [217]:
df_att = df_at[df_at['country'].isin(list_country)]
df_att.head()

,year,type,country,activity,age,species,month,fatal,sex
0,2018,Boating,usa,Paddling,57.0,White Shark,Jun,N,F
2,2018,Unprovoked,usa,Walking,15.0,Bull Shark,May,N,M
3,2018,Provoked,australia,Feeding sharks,32.0,Grey Shark,May,N,M
6,2018,Unprovoked,australia,Surfing,60.0,Unspecified,Apr,N,M
8,2018,Unprovoked,south africa,Paddle-skiing,33.0,White Shark,Apr,N,M


In [218]:
df_att.to_csv('datos/ETL-att-minus.csv')

In [219]:
df_cl = pd.read_csv('datos/clima-clase.csv', index_col=0).reset_index()
df_cl.head()

,index,wind_speed_950mb,wind_speed_900mb,wind_speed_850mb,wind_speed_800mb,wind_speed_750mb,wind_speed_700mb,wind_speed_650mb,wind_speed_600mb,wind_speed_550mb,...,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,0,2,2,3,4,4,5,5,5,5,...,13,15,5,1024,none,0,0,265,2,usa
1,1,2,3,4,5,5,5,5,5,5,...,12,15,5,1024,none,0,0,345,2,usa
2,2,2,2,2,3,3,5,5,5,5,...,13,15,4,1024,none,0,0,260,2,usa
3,3,2,3,3,3,3,4,5,6,6,...,15,10,2,1024,none,0,0,175,2,usa
4,4,2,3,3,3,4,5,6,6,6,...,17,6,3,1024,none,0,0,350,2,usa


In [220]:
df_cl['country'] = df_cl['country'].str.lower()

### Generamos una función que nos permita crear una base de datos:

In [221]:
def make_ddbb(name_ddbb):
  
    mydb = mysql.connector.connect(
      host='127.0.0.1',
      user='root',
      password='AlumnaAdalab')

    mc = mydb.cursor()

    try:
        mc.execute(f"CREATE DATABASE IF NOT EXISTS {name_ddbb};")
        print(mc)
    # en caso de que podamos ejecutar la query devuelvenos un error para saber en que nos estamos equivocando
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

### Aplicamos la función:

In [222]:
db_tiburones = make_ddbb('tiburones')

MySQLCursor: CREATE DATABASE IF NOT EXISTS tiburones;


### Generamos una segunda función para crear las tablas mediante una query.

In [223]:
def insert_table(nombre_bbdd, query):
    
    cnx = mysql.connector.connect(user='root', password=f"AlumnaAdalab",
                                     host='127.0.0.1', database=f"{nombre_bbdd}")
    
    mycursor = cnx.cursor()
 
    try: 
        mycursor.execute(query)
        cnx.commit() 
    # en caso de que podamos ejecutar la query devuelvenos un error para saber en que nos estamos equivocando
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

### Creamos las querys para definir el nombre de las tablas y sus columnas:

In [224]:
attacks = ''' 
    CREATE TABLE IF NOT EXISTS `attacks` (
    `year` INT DEFAULT NULL,
    `type` VARCHAR(50) DEFAULT NULL, 
    `country` VARCHAR(50) DEFAULT NULL,  
    `activity` VARCHAR(100) DEFAULT NULL,  
    `age` FLOAT DEFAULT NULL,
    `species` VARCHAR(50) DEFAULT NULL,
    `month` VARCHAR (45) DEFAULT NULL,
    `fatal` VARCHAR (45) DEFAULT NULL,
    `sex` VARCHAR (45) DEFAULT NULL)
    
    ENGINE = InnoDB;
    '''

In [225]:
climate = '''
       CREATE TABLE IF NOT EXISTS `climate` (
       `country` VARCHAR(45),
       `wind_speed_950mb` FLOAT DEFAULT NULL,
       `wind_speed_900mb`	FLOAT DEFAULT NULL,
       `wind_speed_850mb` FLOAT DEFAULT NULL,
       `wind_speed_800mb`	FLOAT DEFAULT NULL,
       `wind_speed_750mb` FLOAT DEFAULT NULL,
       `wind_speed_700mb`	FLOAT DEFAULT NULL,
       `wind_speed_650mb`	FLOAT DEFAULT NULL,
       `wind_speed_600mb`	FLOAT DEFAULT NULL,
       `wind_speed_550mb`	FLOAT DEFAULT NULL,
       `wind_speed_500mb`	FLOAT DEFAULT NULL,
       `wind_speed_450mb`	FLOAT DEFAULT NULL,
       `wind_speed_400mb` FLOAT DEFAULT NULL,
       `wind_speed_350mb`	FLOAT DEFAULT NULL,
       `wind_speed_300mb`	FLOAT DEFAULT NULL,
       `wind_speed_250mb` FLOAT DEFAULT NULL,
       `wind_speed_200mb`	FLOAT DEFAULT NULL,
       `wind_direction_950mb`	FLOAT DEFAULT NULL,
       `wind_direction_900mb`	FLOAT DEFAULT NULL,
       `wind_direction_850mb`	FLOAT DEFAULT NULL, 
       `wind_direction_800mb`	FLOAT DEFAULT NULL,
       `wind_direction_750mb`	FLOAT DEFAULT NULL,
       `wind_direction_700mb`	FLOAT DEFAULT NULL,
       `wind_direction_650mb`	FLOAT DEFAULT NULL,
       `wind_direction_600mb`	FLOAT DEFAULT NULL,
       `wind_direction_550mb`	FLOAT DEFAULT NULL,
       `wind_direction_500mb` FLOAT DEFAULT NULL,
       `wind_direction_450mb`	FLOAT DEFAULT NULL,
       `wind_direction_400mb`	FLOAT DEFAULT NULL,
       `wind_direction_350mb`	FLOAT DEFAULT NULL,
       `wind_direction_300mb`	FLOAT DEFAULT NULL,
       `wind_direction_250mb`	FLOAT DEFAULT NULL,
       `wind_direction_200mb`	FLOAT DEFAULT NULL,
       `rh_950mb`	FLOAT DEFAULT NULL,
       `rh_900mb`	FLOAT DEFAULT NULL,
       `rh_850mb`	FLOAT DEFAULT NULL,
       `rh_800mb`	FLOAT DEFAULT NULL,
       `rh_750mb`	FLOAT DEFAULT NULL,
       `rh_700mb`	FLOAT DEFAULT NULL,
       `rh_650mb`	FLOAT DEFAULT NULL,
       `rh_600mb`	FLOAT DEFAULT NULL,
       `rh_550mb`	FLOAT DEFAULT NULL,
       `rh_500mb`	FLOAT DEFAULT NULL,
       `rh_450mb`	FLOAT DEFAULT NULL,
       `rh_400mb`	FLOAT DEFAULT NULL,
       `rh_350mb`	FLOAT DEFAULT NULL,
       `rh_300mb`	FLOAT DEFAULT NULL,
       `rh_250mb`	FLOAT DEFAULT NULL,
       `rh_200mb`	FLOAT DEFAULT NULL,
       `timepoint`	FLOAT DEFAULT NULL,
       `cloudcover`	FLOAT DEFAULT NULL,
       `highcloud` FLOAT DEFAULT NULL,
       `midcloud`	FLOAT DEFAULT NULL,
       `lowcloud`	FLOAT DEFAULT NULL,
       `temp2m`	FLOAT DEFAULT NULL,
       `lifted_index` FLOAT DEFAULT NULL,
       `rh2m`	FLOAT DEFAULT NULL,
       `msl_pressure`	FLOAT DEFAULT NULL,
       `prec_amount`	FLOAT DEFAULT NULL,
       `snow_depth`	FLOAT DEFAULT NULL,
       `wind10m.direction`	FLOAT DEFAULT NULL,
       `wind10m.speed` FLOAT DEFAULT NULL)
    
    ENGINE = InnoDB;
       '''

### Llamamos a la función:

In [226]:
insert_table('tiburones', attacks)

In [227]:
insert_table('tiburones', climate)

### Insertamos los datos en nuestras tablas:

In [228]:
for indice, fila in df_att.iterrows():
    # itreamos por el dataframe.
    
    # lo valores que introduciremos serán los del dataframe que estamos iterando, por lo que usaremos los formats de los strings. 
    
    query_x = f"""
            INSERT INTO attacks (year, type, country, activity, age, species, month, fatal, sex)
            VALUES ( "{fila["year"]}", "{fila['type']}", "{fila['country']}", "{fila['activity']}", "{fila['age']}", 
            "{fila['species']}", "{fila['month']}", "{fila['type']}", "{fila['sex']}");
            """
    # una vez definida la query llamamos a la función que nos inserta los datos. 
    insert_table("tiburones", query_x)

### Por optimizar el tiempo y demostrar que sabemos realizar correctamente lo que nos pide el enunciado, hemos optado por insertar los valores en solo seis columnas, puesto que hubieramos perdido mucho tiempo escribiendo el código para insertar los valores de más de 60 columnas.

In [229]:
for indice, fila in df_cl.iterrows():
    # itreamos por el dataframe.
    
    # lo valores que introduciremos serán los del dataframe que estamos iterando, por lo que usaremos los formats de los strings. 
    
    query_y = f"""
            INSERT INTO climate (country, wind_speed_950mb, wind_direction_650mb, rh_600mb, timepoint, cloudcover, highcloud)
            VALUES ( "{fila["country"]}", "{fila['wind_speed_950mb']}", "{fila['wind_direction_650mb']}", "{fila['rh_600mb']}", "{fila['timepoint']}", 
            "{fila['cloudcover']}", "{fila['highcloud']}");
            """
    # una vez definida la query llamamos a la función que nos inserta los datos. 
    insert_table("tiburones", query_y)